# ノード数を変更するスケジュールを設定する--CoursewareHub以外のノードを利用する

---

授業日程に合わせてworkerノードをスケールアウト、スケールインするスケジュールを設定します。

このNotebookではノード数を操作する環境をCoursewareHubを構成するノード以外に構築する手順を実行します。そのためCoursewareHubを構成するノード以外の利用可能なノードが必要となります。追加のノードを利用せずにノード数を操作する環境を構築する場合は、このNotebookのかわりに「95３-workerノードのノード数を変更するスケジュールを設定する.ipynb」を実行してください。


## 概要

このNotebookではCoursewareHubを構成するworkerノードの起動数を、授業日とそれ以外の曜日で変更できるような設定を行います。授業日以外は起動するノード数を減らすことでクラウドプロバイダの課金を抑えるようにします。

次図に、このNotebookでの設定対象を示します。このNotebookではCoursewareHubを構成する以外のノードにノード操作を行う環境を構築します。環境を構築するノード自体は既に存在しているものとします。

![構成](images/cw-954-01.png)

### 前提条件

電源操作を行うための環境を構築するノードの前提条件を以下に示します。

* VC Controllerにアクセスできること
* CoursewareHubを構成する全てのノードにアクセスできること
* このJupyterNotebook環境からアクセスできること
* docker engine, docker composeが利用可能であること
* systemdが利用可能であること

workerノードの電源操作を行う処理はコンテナで実行します。そのため dockerが利用できるよう必要があります。ノードの電源操作はVCP SDKを用いてVC controllerに依頼を行います。そのためVC Controllerにアクセス可能である必要があります。またノードの電源状態以外にdocker swarmのノード状態(availability)の操作も合わせて行います。そのためCoursewareHubのmanagerノードにアクセスできる必要があります。

電源操作の処理はsystemd timerにより起動されます。そのためsystemdが利用可能である必要があります。

### 想定するシナリオ

このNotebookでは、CoursewareHubの利用状況に応じてworkerノードの状態を変更するようにスケジュールを設定します。想定している３つの利用状況を以下に示します。

* 授業時間中
  * 授業開始数十分前から授業終了数十分後まで
  * workerノードを最大限利用する時間帯
* 授業時間後
  * 授業終了数十分後から数時間後まで
  * 利用者が徐々に減少していく時間帯
* 授業時間外
  * 授業時間中、授業時間後以外の時間
  * 利用者が少ない時間帯

それぞれの利用状況に応じて各workerノードの状態を変更します。ノードの状態は、docker swarmノードのavailabilityとVCノードの作成状態を組み合わせたものを定めています。それぞれの状態を以下に示します。

|状態|availability|VCノードの作成状態|
|:---|:---:|:---:|
|利用可能|active|作成|
|新規利用不可|pause|作成|
|利用不可|drain|削除|

docker swarmノードのavailabilityはdocker swarmがそのノードにタスク（コンテナ）を割り当てることができるかを示した設定値となっています。それぞれの値を設定した場合のdocker swarmでの振る舞いを以下に示します。

* active
  * ノードにタスクを割り当てることができる
* pause
  * 新しいタスクを割り当てることはできないが、既存のタスクは引き続き実行される
* drain
  * 実行中のタスクを終了し、他の利用可能なノードで実行するようにスケジュールする

利用状況ごとのworkerノードの設定についての考え方を以下に記します。

#### 授業時間中

![授業時間中](images/cw-951-02.png)

授業時間中は全てのworkerノードを「利用可能」な状態にします。この状態ではworkerノードの計算資源の上限まで、生徒が利用するJupyter環境(single-user Jupyter notebook server)コンテナを起動することができます。

#### 授業時間後
![授業時間中後](images/cw-951-03.png)

授業時間が終了した後、少数の「利用可能」状態のノード以外は新規利用不可に移行します。授業時間から継続して利用している生徒のJupyter環境コンテナは「新規利用不可」にしたノードでも実行し続けますが、新たに起動したコンテナは少数の「利用可能」状態のノードに集約されます。

CoursewareHubのculling機能を有効にすると一定時間アイドルになっているJupyter環境コンテナを自動停止することができます。culling機能を利用することにより「利用可能」状態のノードへの集約を効率的にに行うことができます。

#### 授業時間外
![授業時間外](images/cw-953-04.png)

授業時間外になると「新規利用不可」のノードを「利用不可」に移行します。この状態ではVCノードを削除するのでクラウドプロバイダの課金を抑えられることが期待されます。「利用不可」状態のノードはswarmクラスタから削除されるので、そのノードで実行していたコンテナはいったん終了します。もしその時点で「利用可能」なworkerノードが存在して、かつそのノードの計算資源に空きがある場合は、「利用可能」状態のノードで改めてコンテナが起動されます。

## ノード操作を行う環境の指定

ノード操作コンテナを実行する環境を指定します。ここで指定するのはCoursewareHubを構成する以外のノードになります。

### パラメータの指定

ノード操作コンテナを実行するホスト名（またはIPアドレス）を指定してください。

In [ ]:
# (例)
# nodectl_hostname = 'nodectl.example.org'
# nodectl_hostname = '172.30.2.50'

nodectl_hostname = 

ノードにログインする際のユーザ名を指定してください

In [ ]:
# (例)
# nodectl_user = 'user01'

nodectl_user = 

ノードにログインするためのSSHの秘密鍵ファイルのパスを指定してください。

In [ ]:
# (例)
# nodectl_ssh_private_key = '~/.ssh/id_rsa'

nodectl_ssh_private_key = 

ノードに対する操作にはAnsibleを利用します。そのため対象ノードを新たなAnsibleのグループに登録します。新たに作成するAnsibleのグループ名を指定してください。ここで指定するグループ名は既存のものとは異なる名前を指定してください。

In [ ]:
# (例)
# target_group = 'scaleout'

target_group = 

### チェック

指定されたホストがノード操作コンテナを実行する前提条件を満たしているかの簡単なチェックを行います。

SSHでログインできることを確認します。

In [ ]:
!ssh -i {nodectl_ssh_private_key} \
    -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null \
    {nodectl_user}@{nodectl_hostname} whoami

`~/.ssh/known_hosts`ファイルを更新します。

In [ ]:
!ssh-keyscan {nodectl_hostname} >> ~/.ssh/known_hosts

docker composeが利用できることを確認します。

In [ ]:
!ssh -i {nodectl_ssh_private_key} {nodectl_user}@{nodectl_hostname} \
    docker compose version

systemdが利用できることを確認します。

In [ ]:
!ssh -i {nodectl_ssh_private_key} {nodectl_user}@{nodectl_hostname} \
    systemctl --user status

指定されたグループ名に対応するgroup_varsが存在していないことを確認します。

In [ ]:
! ! test -e group_vars/{target_group}

以下の項目についてはここではチェックしません。

* VC Controllerにアクセスできること
* managerノードにアクセスできること


### ansibleグループの登録

対象ノードをansibleのインベントリに登録します。

In [ ]:
%run scripts/group.py
from pathlib import Path
params = {
    "all": {
        "children": {
            target_group: {
                "hosts": {
                    nodectl_hostname: {
                        "ansible_ssh_private_key_file": str(Path(nodectl_ssh_private_key).expanduser()),
                        "ansible_user": nodectl_user,
                    }
                }
            }
        }
    }
}
path_inventory = update_inventory_yml(params)
!diff -u {path_inventory}.bak {path_inventory} || true

ansibleで対象にアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

## パラメータの指定

ノードの電源操作スケジュールを登録するのに必要となるパラメータを指定します。

### UnitGroup名

操作対象となるCoursewareHubのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

指定されたUnitGroup名をgroup_varsに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    ugroup_name=ugroup_name
)

### VCCアクセストークンの入力

VCノードを操作するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2023/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### IPアドレス

workerノードに割り当てるIPアドレスを指定します。ここでは、workerノードとして起動するノード数の最大数分のIPアドレスを指定する必要があります。

参考のために現在workerノードに割り当てているIPアドレスを確認します。

In [ ]:
ugroup = vcp.get_ugroup(ugroup_name)
unit_worker = ugroup.get_unit('worker')
unit_worker.df_nodes()

workerノードに割り当てるIPアドレスを指定してください。現在のworkerノードに割り当てるIPアドレスと全く異なるアドレスを指定することもできますが、その場合現在のworkerノードが削除された後に新たなノードを作成することになります。また指定するアドレス数は、workerノードとして起動するノード数の最大数に一致している必要があります。

In [ ]:
# (例)
# vcnode_all_ipaddress = [
#    "172.30.2.101",
#    "172.30.2.102",
#    "172.30.2.103",
#]
#
# vcnode_all_ipaddress = [f"172.30.2.{x}" for x in range(101, 110)]

vcnode_all_ipaddress = [
    
]

#### 指定されたパラメータの保存

指定されたパラメータをgroup_varsに保存します。

パラメータの保存を行う前に簡単なチェックを行います。

In [ ]:
from ipaddress import IPv4Address, IPv4Network
%run scripts/group.py

gvars = load_group_vars(ugroup_name)
subnet = IPv4Network(vcp.get_vpn_catalog(gvars['vc_provider']).get('private_network_ipmask'))
for addr in vcnode_all_ipaddress:
    if IPv4Address(addr) not in subnet:
        raise RuntimeError(f"指定されたアドレスは{subnet}に含まれていません: {addr}")

group_varsにパラメータを保存します。

In [ ]:
update_group_vars(
    ugroup_name,
    vcnode_all_ipaddress=vcnode_all_ipaddress,
)

### workerノードの状態変更のスケジュールを指定する

それぞれの利用状況ごとに、どのようなノード状態を設定するかを指定します。

#### 授業日

授業の曜日を指定してください。

In [ ]:
# (例)
# schedule_day_of_week = '月'
# schedule_day_of_week = 'Tue'
# schedule_day_of_week = 'Wednesday'


schedule_day_of_week = 

#### 授業時間中のノード状態

利用状況を「授業時間中」にする開始時間を指定してください。通常は実際の授業開始時刻の３０〜６０分前の時刻を指定します。

In [ ]:
# (例)
# schedule_time_0 = '8:30'

schedule_time_0 = 

「授業時間中」における各ノード状態に設定するノード数を指定します。通常は「利用可能」(active)のノード数にworkerノード数を指定し他の状態には0を指定します。workerノード数の合計は`vcnode_all_ipaddress`に指定したIPアドレス数と一致している必要があります。`vcnode_all_ipaddress`に指定したアドレス数を確認します。

In [ ]:
print(len(vcnode_all_ipaddress))

ノード状態ごとのノード数を指定してください。各状態のノード数の合計がworkerノードに一致するようにしてください。

In [ ]:
# (例)
# node_availability_0 = {
#     "active": 3,    # 利用可能状態のノード数
#     "pause": 0,     # 新規利用不可状態のノード数
#     "drain": 0,       # 利用不可状態のノード数
# }

node_availability_0 = {
    "active": ,
    "pause": ,
    "drain": ,
}

#### 授業時間後のノード状態

利用状況を「授業時間後」にする開始時間を指定してください。通常は実際の授業終了時刻の３０〜６０分後の時刻を指定します。

In [ ]:
# (例)
# schedule_time_1 = '11:00'

schedule_time_1 = 

「授業時間後」における各ノード状態に設定するノード数を指定します。通常は、少数の「利用可能」(active)のノード以外は「新規利用不可」(pause)となるようにノード数を指定します。各状態のノード数の合計がworkerノードに一致するようにしてください。

In [ ]:
# (例)
# node_availability_1 = {
#     "active": 1,    # 利用可能状態のノード数
#     "pause": 2,     # 新規利用不可状態のノード数
#     "drain": 0,       # 利用不可状態のノード数
# }

node_availability_1 = {
    "active": ,
    "pause": ,
    "drain": ,
}

#### 授業時間外のノード状態

利用状況を「授業時間外」にする開始時間を指定してください。通常は実際の授業終了時刻の数時間後の時刻を指定します。

In [ ]:
# (例)
# schedule_time_2 = '13:00'

schedule_time_2 = 

「授業時間外」における各ノード状態に設定するノード数を指定します。通常は、少数の「利用可能」(active)のノード以外は「利用不可」(drain)となるようにノード数を指定します。各状態のノード数の合計がworkerノードに一致するようにしてください。

In [ ]:
# (例)
# node_availability_2 = {
#     "active": 1,    # 利用可能状態のノード数
#     "pause": 0,     # 新規利用不可状態のノード数
#     "drain": 2,       # 利用不可状態のノード数
# }

node_availability_2 = {
    "active": ,
    "pause": ,
    "drain": ,
}

#### 指定されたパラメータの保存

指定されたパラメータをgroup_varsに保存します。

パラメータの保存を行う前に簡単なチェックを行います。

In [ ]:
import re

if schedule_day_of_week.lower() not in [
    "sun", "mon", "tue", "wed", "thu", "fri", "sat",
    "sunday", "monday", "tuesday", "wednesday", "thursday", "friday", "saturday",
    "日", "月", "火", "水", "木", "金", "土",
]:
    raise RuntimeError(f"曜日指定が正しくない: {schedule_day_of_week}")

for x in [schedule_time_0, schedule_time_1, schedule_time_2]:
    if (re.fullmatch(r"(?:[01]?\d|2[0-3]):[0-5]?\d", x, re.A) is None):
        raise RuntimeError(f"時刻指定が正しくない: {x}")

for x in [node_availability_0, node_availability_1, node_availability_2]:
    if sum(x.values()) != len(vcnode_all_ipaddress):
        raise RuntimeError(f"ノード数の合計がworkerノード数と一致していない: {x}")

group_varsにパラメータを保存します。

In [ ]:
%run scripts/group.py
%run scripts/cwh.py
gvars = load_group_vars(ugroup_name)
schedule = [
    {
        "schedule": {
            "day_of_week": schedule_day_of_week.lower(),
            "hour": int(schedule_time_0.split(":", 2)[0]),
            "minute": int(schedule_time_0.split(":", 2)[1]),
        },
        "availability": node_availability_0,
    },
    {
        "schedule": {
            "day_of_week": schedule_day_of_week.lower(),
            "hour": int(schedule_time_1.split(":", 2)[0]),
            "minute": int(schedule_time_1.split(":", 2)[1]),
        },
        "availability": node_availability_1,
    },
    {
        "schedule": {
            "day_of_week": schedule_day_of_week.lower(),
            "hour": int(schedule_time_2.split(":", 2)[0]),
            "minute": int(schedule_time_2.split(":", 2)[1]),
        },
        "availability": node_availability_2,
    },
]
teacher_accounts = [get_username_from_mail_address(email)  for email in gvars["teacher_email_list"]]
update_group_vars(
    target_group,
    vcnode_schedule=schedule,
    vcnode_all_ipaddress=vcnode_all_ipaddress,
    teacher_accounts=teacher_accounts,
    manager_ipaddress=gvars["manager_ipaddress"],
    master_fqdn=gvars["master_fqdn"],
    registry_pass=gvars["registry_pass"],
    ssh_private_key_path=gvars["ssh_private_key_path"],
)
if "nfs_ipaddress" in gvars:
    update_group_vars(
        target_group,
        nfs_ipaddress=nfs_ipaddress,
    )
update_group_vars(
    ugroup_name,
    schedule="scale-out",
)

### 資材を配置するディレクトリの指定

電源操作を行うコンテナ資材を配置するディレクトリを指定します。

In [ ]:
# (例)
# vcnode_ctl_dir = f'/home/{nodectl_user}/scale-out

vcnode_ctl_dir = f'/home/{nodectl_user}/scale-out'

指定された値をgroup_varsに保存します。

In [ ]:
update_group_vars(
    target_group,
    vcnode_ctl_dir=vcnode_ctl_dir
)

### dockerに関するパラメータの指定

構築環境における`docker`コマンドのパスを確認します。

In [ ]:
out = !ansible {target_group} -a 'which docker' 2> /dev/null
path_docker_cli = out[1]
print(path_docker_cli)

docker swarmノードのavailability設定を変更するにはswarmクラスタのmanagerノードのdocker engineにアクセスする必要があります。電源操作を行うノードからmanagerノードのdocker engineにアクセスするためにSSHを利用する（参照:[Use SSH to protect the Docker daemon socket](https://docs.docker.com/engine/security/protect-access/#use-ssh-to-protect-the-docker-daemon-socket)）。managerノードのIPアドレスからDOCKER_HOSTに相当する値を導出します。

In [ ]:
%run scripts/group.py
manager_addr = load_group_var(ugroup_name, "manager_ipaddress")
manager_docker_host = f"ssh://vcp@{manager_addr}"
print(manager_docker_host)

group_vars に保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    path_docker_cli=path_docker_cli,
    manager_docker_host=manager_docker_host,
)

## 資材の配置

電源操作を行うノードに必要となる資材を配置します。

実際の変更を行う前にチェックモードで資材を配置するplaybookを実行します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
import json

with TemporaryDirectory() as work_dir:
    vars_path = Path(work_dir) / "params.json"
    with vars_path.open(mode="w") as f:
        json.dump({"vcc_access_token": vcc_access_token}, f)
    !ansible-playbook -CDv -l {target_group} -e @{str(vars_path)} playbooks/setup-scaleout-user.yml || true

実際に資材の配置を行います。

In [ ]:
with TemporaryDirectory() as work_dir:
    vars_path = Path(work_dir) / "params.json"
    with vars_path.open(mode="w") as f:
        json.dump({"vcc_access_token": vcc_access_token}, f)
    !ansible-playbook -l {target_group} -e @{str(vars_path)} playbooks/setup-scaleout-user.yml

## タイマーの開始

設定ファイルを配置したsystemdタイマーを開始してノードの更新処理がスケジュールされるようにします。

In [ ]:
!ansible {target_group} -m systemd -a \
    'daemon_reload=true name=cwh-worker-node.timer enabled=true state=restarted scope=user'

タイマーの一覧を表示して登録されたことを確認します。

In [ ]:
!ansible {target_group} -a 'systemctl --user list-timers'

サービスのログを確認します。

In [ ]:
!ansible {target_group} -a 'journalctl --user -u cwh-worker-node -n 30'

## CoursewareHubのculling設定

CoursewareHubを構成するJupyterHubコンテナに環境変数を設定することでculling設定を有効にすることができます。「CoursewareHubのセットアップ.ipynb」でCoursewareHub環境を構築した時にculling機能を有効にしていない場合は、必要に応じてこの章を実行してください。

### パラメータの指定

アイドル状態のsingle-userサーバーを自動的に停止する機能に関するパラメータを指定します。

ここで指定できるパラメータを以下に示します。

|項目 | 説明 | デフォルト値|
|:---|:---|--:|
|cull_server              | アイドル状態のNotebookサーバーの停止を有効／無効化 (yes/no/0/1)| no |
|cull_server_idle_timeout | アイドル状態のNotebookサーバーの停止までのタイムアウト時間(秒)     | 600 |
|cull_server_max_age      | アイドル状態でなくてもNotebookサーバーを停止するまでの時間(秒)     | 0 |
|cull_server_every       | Notebookのアイドル状態のタイムアウトのチェック間隔(秒)             | 0 |

各パラメータの値を以下のセルで指定してください。デフォルト値のままで良いパラメータはセルを実行せずにスキップしてください。

In [ ]:
# (例)
# cull_server = "yes"       # 自動停止を有効にする
# cull_server = "no"        # 自動停止を無効にする

cull_server = "yes"

In [ ]:
# (例)
# cull_server_idle_timeout = 10 * 60               # １０分間
# cull_server_idle_timeout = 12 * 60 * 60      # 12時間

cull_server_idle_timeout = 

In [ ]:
# (例)
# cull_server_max_age = 0
# cull_server_max_age = 24 * 60 * 60      # 24時間

cull_server_max_age = 

In [ ]:
# (例)
# cull_server_every = 0
# cull_server_every = 60

cull_server_every = 

#### パラメータの保存

ここまで指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

jupyterhub_param_names = [
    'cull_server',
    'cull_server_idle_timeout',
    'cull_server_max_age',
    'cull_server_every',
]

for name in jupyterhub_param_names:
    if name in vars():
        gvars[name] = vars()[name]
    elif 'name' in gvars:
        del(gvars['name'])

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)
    
!cat group_vars/{ugroup_name}

### docker-compose.ymlの配置

構築環境にculling設定を追加したdocker-compose.ymlを配置します。

まずはチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {ugroup_name}_manager -CDv playbooks/deploy-docker-compose.yml || true

実際に`docker-compose.yml`の配置を行います。

In [ ]:
!ansible-playbook -l {ugroup_name}_manager playbooks/deploy-docker-compose.yml

### コンテナの更新

指定されたパラメータに更新された`docker-compose.yml`でコンテナを起動し直します。

コンテナ設定の更新を行います。culling設定を変更している場合はjupyterhubコンテナが起動され直します。

In [ ]:
!ansible {ugroup_name}_manager -a 'chdir={{{{compose_dir}}}} \
    docker stack deploy -c docker-compose.yml coursewarehub'

コンテナの起動状態を確認します。

In [ ]:
!ansible {ugroup_name}_manager -a 'docker service ls'